In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def sensitivityshock2(df,
                      model,
                      shockvar,
                      shockrange=(0, 0.1, 0.2),
                      bounds=None):                      #  ← NEW
    """
    df         : modelling data-frame
    model      : fitted statsmodels GLM / OLS / Logit …
    shockvar   : list of variable names to shock
    shockrange : iterable of % shocks (0.1 == +10 %)
    bounds     : dict {var: [lower, upper]}
                 • lower / upper can be None   (⇒ no bound)
                 • omit var altogether for no bounds
                 Example: {'util':[0,1], 'age':[18,None], 'inc':None}
    """

    if bounds is None:
        bounds = {}

    # ─────────────────── helper ────────────────────
    def clamp(series, var):
        """clip Series to bounds[var] if supplied"""
        if var not in bounds or bounds[var] is None:
            return series
        lb, ub = (bounds[var] + [None, None])[:2]   # tolerate [lb] or [lb,ub]
        return series.clip(lower=lb, upper=ub)
    # ───────────────────────────────────────────────

    # Down-sample huge frames for speed (unchanged)
    if df.shape[0] > 1_000_000:
        df = df.sample(1_000_000, random_state=42)

    summary = pd.DataFrame()

    # figure out “direction” lists (same logic you already had)
    negative = list(
        (set(i.split("_T")[0]
             .replace("sp_imp_", "")
             for i in model.params.loc[model.params[0].index
                                        if "const" not in i])
         ).union(set(i for i in model.params[0].index
                     if "const" not in i))
        ).intersection(shockvar)
    )
    positive = [i for i in shockvar if i not in negative]
    all_var  = model.params.index.tolist()[1:]      # drop intercept

    # ───────────── one-at-a-time shocks ────────────
    for var in shockvar:
        if var not in df.columns:
            print(f"{var} not in df → skipped")
            continue

        for shock in shockrange:
            factor = (1 - shock) if var in negative else (1 + shock)

            nonmacro        = df.copy()
            nonmacro[var]   = clamp(nonmacro[var] * factor, var)

            score           = model.predict(sm.tools.add_constant(nonmacro[all_var]))
            row             = pd.DataFrame({"Score":  score.mean(),
                                            "VAR":    var,
                                            "Shock":  shock}, index=[0])
            summary = pd.concat([summary, row], axis=0, sort=False)

    # ───────────── all-vars-at-once shocks ─────────
    for shock in shockrange:
        nonmacro = df.copy()

        for v in positive:
            if "_T" not in v:
                nonmacro[v] = clamp(nonmacro[v] * (1 + shock), v)

        for v in negative:
            if "_T" not in v:
                nonmacro[v] = clamp(nonmacro[v] * (1 - shock), v)

        score = model.predict(sm.tools.add_constant(nonmacro[all_var]))
        row   = pd.DataFrame({"Score": score.mean(),
                              "VAR":   "ALL_VARS",
                              "Shock": shock}, index=[0])
        summary = pd.concat([summary, row], axis=0, sort=False)

    # tidy pivot (unchanged)
    pivot_df              = summary.pivot(index="VAR",
                                          columns="Shock",
                                          values="Score")
    pivot_df.columns.name = None
    return pivot_df

In [ ]:
import numpy as np
import pandas as pd

def model_shock(
    df: pd.DataFrame,
    model,                              # fitted statsmodels GLM
    shocks: list,                       # e.g. [-0.2, -0.1, 0, 0.1, 0.2]
    vars_bounds: dict,                  # {'var1':[0,1], 'var2':[None,5], ...}
    agg_func=np.mean,                   # summary statistic (default mean)
    include_all=True                    # add simultaneous-shock rows
) -> pd.DataFrame:
    """
    One-at-a-time AND all-at-once percentage shocks on selected variables.
    Returns tidy table with impact vs baseline.
    """
    # — baseline —
    baseline_stat = agg_func(model.predict(df))

    rows = []
    var_names = list(vars_bounds.keys())

    def apply_bounds(series, lb, ub):
        if lb is not None:
            series = series.clip(lower=lb)
        if ub is not None:
            series = series.clip(upper=ub)
        return series

    for s in shocks:
        # ---------- one-at-a-time ----------
        for var in var_names:
            dfi = df.copy()
            lb, ub = (vars_bounds[var] + [None, None])[:2] if isinstance(vars_bounds[var], list) else (None, None)
            dfi[var] = apply_bounds(dfi[var] * (1 + s), lb, ub)

            stat       = agg_func(model.predict(dfi))
            delta      = stat - baseline_stat
            pct_change = delta / baseline_stat if baseline_stat else np.nan
            rows.append({'variable': var, 'shock': s, 'stat': stat,
                         'delta': delta, 'pct_change': pct_change})

        # ---------- all-vars-at-once ----------
        if include_all:
            dfi = df.copy()
            for var in var_names:
                lb, ub = (vars_bounds[var] + [None, None])[:2] if isinstance(vars_bounds[var], list) else (None, None)
                dfi[var] = apply_bounds(dfi[var] * (1 + s), lb, ub)

            stat       = agg_func(model.predict(dfi))
            delta      = stat - baseline_stat
            pct_change = delta / baseline_stat if baseline_stat else np.nan
            rows.append({'variable': 'ALL_VARS', 'shock': s, 'stat': stat,
                         'delta': delta, 'pct_change': pct_change})

    return pd.DataFrame(rows)